## Chile properties SQL practice 

### Use all the tables of this database

In [ ]:
SELECT * 
FROM `properati-data-public.properties_cl.*`
LIMIR 10;

### Find the count and share of each property type listings in 2015

In [ ]:
# method 1 - using group by 

SELECT property_type, 
       count(*) as property_cnt, 
       count(*)/sum(count(*)) over () as property_percent
FROM `properati-data-public.properties_cl.*`
WHERE extract(year from created_on) = 2015
GROUP BY property_type;


In [ ]:
# method 2 - using window functions only
SELECT  distinct 
        property_type, 
        count(*) over(partition by property_type) as property_cnt, 
        count(*) over(partition by property_type) / count(*) over () as percent
FROM `properati-data-public.properties_cl.*`
WHERE extract(year from created_on) = 2015;
# - use of distinct neglects the need for the group by clause

### Find the share of property_type listings in 2015, order by highest to lowest share.

In [ ]:
# method 1 - using group by

SELECT property_type, 
       count(*) as property_cnt, 
       100 * count(*)/ sum(count(*)) over () as property_percent
FROM `properati-data-public.properties_cl.*`
WHERE extract(year from created_on) = 2015
GROUP BY property_type
ORDER BY property_percent desc;

# - notice: the count(*) at the numerator prior the window function acts according to the groupby clause.

In [ ]:
# method 2 - using distinct and window functions

SELECT distinct 
       property_type, 
       count(*) over(partition by property_type) as property_cnt, 
       100 * count(*) over(partition by property_type) / count(*) over () as property_percent
FROM `properati-data-public.properties_cl.*`
WHERE extract(year from created_on) = 2015
ORDER BY property_percent desc;


### Find the share of apartment listings in 2015

In [ ]:
# method 1 - subquery in the select statement

SELECT count(*) / (
                    SELECT count(*)
                    FROM `properati-data-public.properties_cl.*`
                    WHERE extract(year from created_on) = 2015
                    ) as apt_portion
FROM `properati-data-public.properties_cl.*`
WHERE extract(year from created_on) = 2015 and property_type = 'apartment';


In [ ]:
# method 2 - using cte

with cte as 
    (
    SELECT property_type, 
           count(*) as property_cnt
    FROM `properati-data-public.properties_cl.*`
    WHERE extract(year from created_on) = 2015
    GROUP BY property_type
    )

SELECT property_cnt	 / (SELECT sum(property_cnt) FROM cte) as apt_portion
FROM cte
WHERE property_type = 'apartment';


In [ ]:
# method 3 -  using case statement - no subquery nor cte required.

SELECT count(case when property_type = 'apartment' then 'property_type' end) / count(*) as apt_portion
from `properati-data-public.properties_cl.*`
WHERE extract(year from created_on) = 2015;


### Find the number of houses with price of less then 2000 for each month in 2015

In [ ]:
# method 1 

SELECT cheap_tbl.month, cheap_apts / tot_apts as cheap_portion
FROM
    (
     
    (SELECT extract(month from created_on) as month, count(*) as cheap_apts
     FROM `properati-data-public.properties_cl.*` 
     WHERE extract(year from created_on) = 2015 and property_type = 'apartment' and price < 2000
     GROUP BY month) cheap_tbl

    left join 
    
    (SELECT extract(month from created_on) as month, count(*) as tot_apts
     FROM `properati-data-public.properties_cl.*`
     WHERE extract(year from created_on) = 2015 and property_type = 'apartment'
     GROUP BY month) tot_tbl

    on cheap_tbl.month = tot_tbl.month 
    )

ORDER BY month;

In [ ]:
# method 2- do the same, but insert the two subq into cte 

with cte as (
    SELECT * 
    FROM (SELECT extract(month from created_on) as month_cheap, count(*) as cheap_apts
          FROM `properati-data-public.properties_cl.*` 
          WHERE extract(year from created_on) = 2015 and property_type = 'apartment' and price < 2000
          GROUP BY month_cheap) cheap_tbl

          left join 
    
         (SELECT extract(month from created_on) as month_total, count(*) as tot_apts
          FROM `properati-data-public.properties_cl.*`
          WHERE extract(year from created_on) = 2015 and property_type = 'apartment'
          GROUP BY month_total) tot_tbl

          on cheap_tbl.month_cheap = tot_tbl.month_total 
        )

SELECT month_cheap as month, cheap_apts / tot_apts 
FROM cte
ORDER BY month;

### test a case when, with aggregate: For each month in 2015, find the percentage of listings of type house 


In [ ]:
select  extract(month from created_on) as month, 
        count(case when property_type = 'apartment' then 'apt' end) as apt_cnt,
        count(*) as total_listings, 
        count(case when property_type = 'apartment' then 'apt' end) * 100 / count(*)  as apt_precentage
from `properati-data-public.properties_cl.*`
where extract(year from created_on) = 2015
group by month
order by month;


### Buckets: For listed rentel apartments in chile: calculate how many are sold above and below the average price 

In [ ]:
with cte as (
             select property_type, 
             price, 
             case when price > (select avg(price) from `properati-data-public.properties_cl.*` where property_type = 'apartment' and operation = 'rent') then 'above_average' else 'below_average' end as ratio
from `properati-data-public.properties_cl.*`
where property_type = 'apartment' and operation = 'rent'
) 

select ratio, count(ratio) / (select count(*) from cte) as relative_ratio 
from cte 
group by ratio;